In [54]:
#  min_max_normalization_of_dict is a function that normalizes the values
#  in a dict parameter dict_jd_feature_weights and returns a new dict
#  that has normalized values
#  It is assumed that the values in dict_jd_feature_weights will need 
#  to be standardized using min-max normalization as described in
#  https://g.co/bard/share/b244b5fe099b
#  The beauty of min-max mormalization is that it is idempotent 

def min_max_normalization_of_dict(dict_jd_feature_weights):
    normalized_dict_jd_feature_weights = {}

    #find min and max needed for normalization
    min_val = float('inf')
    max_val = float('-inf')
    sum_total = 0
    for key in dict_jd_feature_weights:
        if (dict_jd_feature_weights[key] > max_val):
            max_val = dict_jd_feature_weights[key]
        if (dict_jd_feature_weights[key] < min_val):
            min_val = dict_jd_feature_weights[key]
        sum_total += dict_jd_feature_weights[key]
    
        #populate normalized weights
    for key in dict_jd_feature_weights:
        normalized_dict_jd_feature_weights[key] = \
            dict_jd_feature_weights[key] / sum_total
            
    return normalized_dict_jd_feature_weights

In [62]:
#scoring_comparison is a function that takes two parameters
# 1. dict_jd_feature_weights which is a dictionary containing 
#    keys as desired in the job description and corresponding
#    values that would be weights
#    example
#      {   #key  :  #value
#          java  :  8
#          git   :  7
#          agile :  4
#          springboot : 9
#          sql   :  5
#          .net :  2
#          c#   :  3
#          total_experience_fresher : 0 
#          total_experience_2_to_5_yrs :  0
#          total_experience_5_to_8_yrs :  8
#      }
# 2. set_cv_features which takes in a set of features from the resume
#    example
#     {
#          java
#          git  
#          agile 
#          springboot 
#          sql  
#          .net 
#     }
#  The function returns a score between 1 through 5

def scoring_comparison(dict_jd_feature_weights, set_jd_mandatory_features, set_cv_features):
    return_score = 0
    multiplier = 5
    
    #new dict for normalized dict_jd_feature_weights
    normalized_dict_jd_feature_weights = \
       min_max_normalization_of_dict(dict_jd_feature_weights)
    
    cv_satisfies_mandatory_features = False
    features_matched = set_jd_mandatory_features.intersection(set_cv_features)
    if (len(features_matched) == len(set_jd_mandatory_features)):
        for cv_feature in set_cv_features:
            if (cv_feature in normalized_dict_jd_feature_weights):
                return_score += normalized_dict_jd_feature_weights[cv_feature]
    return return_score * multiplier
    

In [83]:
import unittest

class MyTest(unittest.TestCase):
    def test_all_match_gives_full_score(self):
        dict_jd_feature_weights = {'java':10, 'git':7, 'agile':4, 'springboot':9, 'sql':5,'.net':2, 'c#':3, \
                                    'total_experience_5_to_8_yrs' :  8}
        set_jd_mandatory_features = {'java', 'sql'}
        set_cv_features = {'java', 'git', 'agile', 'springboot', 'sql', 'c#', '.net', \
                                    'total_experience_5_to_8_yrs'}
        score = scoring_comparison(dict_jd_feature_weights, set_jd_mandatory_features, set_cv_features)
        self.assertEqual(score, 5)
    
    def test_doesnt_match_mandatory_requirements_score(self):
        dict_jd_feature_weights = {'java':10, 'git':7, 'agile':4, 'springboot':9, 'sql':5,'.net':2, 'c#':3, \
                                    'total_experience_5_to_8_yrs' :  8}
        set_jd_mandatory_features = {'java', 'sql'}
        set_cv_features = {'java', 'git', 'agile', 'springboot', 'c#', '.net', \
                                    'total_experience_5_to_8_yrs'}
        score = scoring_comparison(dict_jd_feature_weights, set_jd_mandatory_features, set_cv_features)
        self.assertEqual(score, 0)
        
    def test_partial_match_score(self):
        dict_jd_feature_weights = {'java':10, 'git':8, 'agile':1, 'springboot':10, 'sql':5,'.net':1, 'c#':1, \
                                    'total_experience_5_to_8_yrs' :  4}
        set_jd_mandatory_features = {'java', 'sql'}
        set_cv_features = {'java', 'git', 'agile', 'c#', 'sql'}
        score = scoring_comparison(dict_jd_feature_weights, set_jd_mandatory_features, set_cv_features)
        self.assertEqual(score, 3.125)
        
    def test_only_mandatory_requirements_score(self):
        dict_jd_feature_weights = {'java':10, 'git':7, 'agile':4, 'springboot':9, 'sql':10,'.net':2, 'c#':3, \
                                    'total_experience_5_to_8_yrs' :  8}
        set_jd_mandatory_features = {'java', 'sql'}
        set_cv_features = {'java', 'sql', 'springboot'}
        score = scoring_comparison(dict_jd_feature_weights, set_jd_mandatory_features, set_cv_features)
        self.assertEqual(score, 2.735849056603774)
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # t = MyTest()
    # t.ut_all_match_gives_full_score()

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
